In [1]:
import os
import pandas as pd

from tqdm import tqdm

#セットアップ
tqdm.pandas()

import sys
sys.path.append("/kaggle/src")
from kaggle_metrics.kaggle_kl_div import compute_each_score

In [2]:
oof_path = "/kaggle/working/exp000_notrepeat_inputs/tf_efficientnet_b0_ns_oof_df_exp000_notrepeat_inputs_stage1.csv"
oof_df = pd.read_csv(oof_path)

In [3]:
# pred_*をpatient_idごとの平均値との平均値に置き換える
# pred_cols = [col for col in oof_df.columns if "pred_" in col]
# vote_cols = [col for col in oof_df.columns if "_vote" in col if not col.startswith("pred_")]
# print(vote_cols)
# patient_idごとの平均値
# oof_df[vote_cols]
pred_cols = [col for col in oof_df.columns if "pred_" in col]
oof_df[pred_cols]

,pred_seizure_vote,pred_lpd_vote,pred_gpd_vote,pred_lrda_vote,pred_grda_vote,pred_other_vote
0,0.711850,0.059821,0.031907,0.030819,0.019863,0.145740
1,0.008218,0.037198,0.074058,0.054805,0.541758,0.283964
2,0.258192,0.184464,0.087159,0.113383,0.082747,0.274054
3,0.028503,0.032555,0.044432,0.127075,0.377061,0.390374
4,0.030377,0.082431,0.343688,0.005772,0.008369,0.529363
...,...,...,...,...,...,...
20178,0.057157,0.013913,0.006017,0.049497,0.120818,0.752598
20179,0.106545,0.088707,0.011961,0.169083,0.066964,0.556740
20180,0.081017,0.028448,0.009771,0.117948,0.395312,0.367504
20181,0.101816,0.119881,0.020351,0.162056,0.141520,0.454377


In [4]:
patient_mean = oof_df.groupby("patient_id")[pred_cols].mean().reset_index()
display(patient_mean)

,patient_id,pred_seizure_vote,pred_lpd_vote,pred_gpd_vote,pred_lrda_vote,pred_grda_vote,pred_other_vote
0,56,0.042550,0.040713,0.279435,0.018188,0.054166,0.564948
1,105,0.187922,0.035606,0.021480,0.108159,0.292242,0.354592
2,149,0.084435,0.029937,0.036115,0.104077,0.585805,0.159631
3,195,0.065198,0.080935,0.042464,0.054012,0.138146,0.619245
4,198,0.234771,0.058078,0.014185,0.028607,0.017945,0.646415
...,...,...,...,...,...,...,...
1945,65378,0.052732,0.064737,0.057499,0.078594,0.132680,0.613758
1946,65430,0.121903,0.087396,0.022866,0.019647,0.022603,0.725585
1947,65442,0.064250,0.093905,0.037733,0.091004,0.037665,0.675442
1948,65480,0.050432,0.019254,0.015227,0.067602,0.488751,0.358734


In [5]:
display(oof_df[["patient_id"] + pred_cols].head())

,patient_id,pred_seizure_vote,pred_lpd_vote,pred_gpd_vote,pred_lrda_vote,pred_grda_vote,pred_other_vote
0,42516,0.711850,0.059821,0.031907,0.030819,0.019863,0.145740
1,2944,0.008218,0.037198,0.074058,0.054805,0.541758,0.283964
2,33127,0.258192,0.184464,0.087159,0.113383,0.082747,0.274054
3,2338,0.028503,0.032555,0.044432,0.127075,0.377061,0.390374
4,47284,0.030377,0.082431,0.343688,0.005772,0.008369,0.529363


In [6]:
# patient_idごとの平均値との平均値に置き換える
oof_df = oof_df.merge(patient_mean, on="patient_id", how="left", suffixes=("", "_patient_mean"))

for col in pred_cols:
    mean_col = f"{col}_patient_mean"
    oof_df[col] = (oof_df[col] + oof_df[mean_col]) / 2
oof_df = oof_df.drop(columns=[f"{col}_patient_mean" for col in pred_cols])

In [7]:
display(oof_df[["patient_id"] + pred_cols].head())

,patient_id,pred_seizure_vote,pred_lpd_vote,pred_gpd_vote,pred_lrda_vote,pred_grda_vote,pred_other_vote
0,42516,0.717253,0.054910,0.029874,0.029159,0.020933,0.147871
1,2944,0.007298,0.036807,0.075165,0.049556,0.491330,0.339845
2,33127,0.306406,0.173750,0.077903,0.093465,0.077522,0.270954
3,2338,0.035322,0.060213,0.115874,0.119464,0.284861,0.384265
4,47284,0.090385,0.058260,0.202437,0.011777,0.014654,0.622487


In [8]:
oof_df["score"] = oof_df.progress_apply(compute_each_score, axis=1)

100%|██████████| 20183/20183 [03:16<00:00, 102.90it/s]


In [9]:
oof_df["score"].mean()

0.6842664423425536

In [10]:
for fold in range(5):
    print(f" === fold {fold} ===")
    fold_oof_df = oof_df[oof_df["fold"] == fold]
    display(fold_oof_df.groupby("expert_consensus")["score"].mean())

 === fold 0 ===


expert_consensus
GPD        0.736829
GRDA       0.832275
LPD        0.611936
LRDA       1.555290
Other      0.458807
Seizure    0.702056
Name: score, dtype: float64

 === fold 1 ===


expert_consensus
GPD        0.703273
GRDA       0.884285
LPD        0.762709
LRDA       1.233772
Other      0.487208
Seizure    0.720294
Name: score, dtype: float64

 === fold 2 ===


expert_consensus
GPD        0.630878
GRDA       0.897810
LPD        0.810052
LRDA       1.203980
Other      0.455170
Seizure    0.725835
Name: score, dtype: float64

 === fold 3 ===


expert_consensus
GPD        0.769788
GRDA       1.186990
LPD        0.745874
LRDA       1.530588
Other      0.411410
Seizure    0.722439
Name: score, dtype: float64

 === fold 4 ===


expert_consensus
GPD        0.528686
GRDA       1.351777
LPD        0.809562
LRDA       1.496476
Other      0.474466
Seizure    0.642430
Name: score, dtype: float64